**Name:** Cathaoir Agnew

**ID:** 16171659

#Task 1 
**********************************

In [1]:
import pandas as pd
 
!wget https://norvig.com/ngrams/count_1w.txt #unigram corpus
!wget https://norvig.com/ngrams/count_2w.txt #bigram corpus
 
filePath1 = "/content/count_1w.txt"
filePath2 = "/content/count_2w.txt"
 
unigrams_df = pd.read_csv(filePath1,sep='\t',header=None, names=['unigram','count'])
bigrams_df = pd.read_csv(filePath2,sep='\t',header=None, names=['bigram','count'])
 
print(f'Number of unigrams: {unigrams_df.size} Number of Bigrams: {bigrams_df.size}')
 
display(unigrams_df.head(100))
display(bigrams_df.head(100))

--2020-11-12 11:35:31--  https://norvig.com/ngrams/count_1w.txt
Resolving norvig.com (norvig.com)... 158.106.138.13
Connecting to norvig.com (norvig.com)|158.106.138.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4956241 (4.7M) [text/plain]
Saving to: ‘count_1w.txt.3’

count_1w.txt.3      100%[===================>]   4.73M  21.1MB/s    in 0.2s    

2020-11-12 11:35:31 (21.1 MB/s) - ‘count_1w.txt.3’ saved [4956241/4956241]

--2020-11-12 11:35:31--  https://norvig.com/ngrams/count_2w.txt
Resolving norvig.com (norvig.com)... 158.106.138.13
Connecting to norvig.com (norvig.com)|158.106.138.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5566017 (5.3M) [text/plain]
Saving to: ‘count_2w.txt.3’

count_2w.txt.3      100%[===================>]   5.31M  22.6MB/s    in 0.2s    

2020-11-12 11:35:31 (22.6 MB/s) - ‘count_2w.txt.3’ saved [5566017/5566017]

Number of unigrams: 666666 Number of Bigrams: 572716


,unigram,count
0,the,23135851162
1,of,13151942776
2,and,12997637966
3,to,12136980858
4,a,9081174698
...,...,...
95,like,520585287
96,service,519537222
97,x,508609523
98,than,502609275


,bigram,count
0,0Uplink verified,523545
1,0km to,116103
2,1000s of,939476
3,100s of,539389
4,100th anniversary,158621
...,...,...
95,24th of,327460
96,25th anniversary,261023
97,25th of,397735
98,26th of,271707


# Task 2 
***************************

In [2]:
def probability(sentence , verbose = False):

  # probability ( wi | wi-1 )  =  count ( wi , wi-1 ) / count (wi-1)

  # now need to split up sentence into its individual words, which can be performed by using split() , default split operator is white space 
  split_sentence = sentence.split()

  # print statements
  if verbose == True:
    print("Sentence:" ,split_sentence)
    print("")
  sent_probabilty = 1

  for i in range(len(split_sentence)-1):

    # putting strings into variables to make it easier to look at
    uni_string = split_sentence[i]
    bi_string = str(split_sentence[i] + " " + split_sentence[i+1])

    # putting value of counts into variables 
    uni_gram_count_val = int(unigrams_df.loc[unigrams_df.unigram == uni_string , 'count'].values)
    bi_gram_count_val = int(bigrams_df.loc[bigrams_df.bigram == bi_string , 'count'].values)

    bigram_prob = bi_gram_count_val / uni_gram_count_val

    if verbose == True:
      print(f'Unigram count for "{uni_string}" = {uni_gram_count_val} ')
      print(f'Bigram count for "{bi_string}" = {bi_gram_count_val}')
      print(f'Bigram Probability for "{bi_string}" = {bigram_prob}')

      print("--------------------------------------------------")
      print("")
  
    sent_probabilty *= bigram_prob 

  
  if verbose == True:
    print(f'Sentence probailty for "{sentence}" = {sent_probabilty:.20f}')
    print("")
    print("-----------------------------------------------------")

  return sent_probabilty

probability("i love you" , verbose = True) > probability("i hate you" , verbose = True)

Sentence: ['i', 'love', 'you']

Unigram count for "i" = 3086225277 
Bigram count for "i love" = 3979312
Bigram Probability for "i love" = 0.001289378332053626
--------------------------------------------------

Unigram count for "love" = 201063526 
Bigram count for "love you" = 5428714
Bigram Probability for "love you" = 0.02699999402178991
--------------------------------------------------

Sentence probailty for "i love you" = 0.00003481320725727335

-----------------------------------------------------
Sentence: ['i', 'hate', 'you']

Unigram count for "i" = 3086225277 
Bigram count for "i hate" = 876611
Bigram Probability for "i hate" = 0.0002840398614232463
--------------------------------------------------

Unigram count for "hate" = 21274675 
Bigram count for "hate you" = 504048
Bigram Probability for "hate you" = 0.023692394830943365
--------------------------------------------------

Sentence probailty for "i hate you" = 0.00000672958454456599

---------------------------------

True

# Task 3 
****************************************

In [3]:
import numpy as np

def ShannonVisualization(word, n_sentence = 5):

  # n_sentence is the number of words you will allow in a sentence if no exit clause is hit.
  # word_counter is the number of current words making up the sentence

  word = word 
  stopper = True
  new_dictionary = {}
  sentence = word
  word_counter = 1

  # will run this while stopper == True , that is as long as we get a match of the new word with 
  # a bi-gram starting with this new word. Example "men who", now check does a bi-gram start with "who"
  # if there is no bi-gram starting with "who" , stopper = False, and we stop iterating
  # also a word_counter to stop infinte loops, creating infitinly long sentences, this number of n_sentence is user defined.

  while stopper == True and word_counter < n_sentence :

    print(f'Word = {word}') 
    new_dictionary = {}
    # initalize flag 
    found_flag = False

    # loop through each of the bigrams in the dataframe
    for i in bigrams_df.bigram:

      # split the string of the bigram, as it is stored as 1 string
      split_bigram = i.split()
      # since we want to match on the first gram of the bi-gram, we match on split_bigram[0]
      match_check = split_bigram[0]
      
      if word == match_check:
        # this is a flag check, checks to see if we have found a match, if we have found a match , found_flag is set to true
        if found_flag != True:
          found_flag = True

        # creates a dictionary with the key as the bi-gram and the value the probabilities for each of the matches

        new_dictionary[i] = probability(i)
        #new_dictionary[i] = int(bigrams_df.loc[bigrams_df.bigram == i , 'count'].values)

        # technically you would not need to calculate the probabilites, since wi-1 is a constant term in the probability calculation. 
        # And therefore the highest probability would actually just be the highest number of the counts of the bi-grams you are considering.
        # However since the algorithm in Dan Jurafsky states Shannon Viz Method uses probabilities, I said I would in fact calculate the probabilites
        # even if this is less efficient, as it's technically correct.

        
        
    #print(new_dictionary)

    # if we failed to match our new word to the start of a bi-gram, found flag will remain false
    # we then make stopper = false, stopping the while loop
    if found_flag == False:
      stopper = False 
      print("Word had no match")
  
    # else statement allows code to run fine if there was a match
    else:

      # getting the max key & val 
      max_key = None
      max_val = None

      for key, val in new_dictionary.items():
        if max_val is None or val > max_val:
          max_val = val
          max_key = key

      #print("Max Key:", max_key)
      #print("Max Val:", max_val)

      print(f'Most frequent bi-gram = {max_key}')

      # new word will be the second part of the bigram
      word = max_key.split()[1]
      # increase counter by 1, as a word will be added
      word_counter += 1

    # add word to the string, once its a valid one
    if found_flag == True:
      sentence += " " + (word)

  # finally print out the sentence
  print("\n",sentence)

In [4]:
ShannonVisualization("man" , 5)

Word = man
Most frequent bi-gram = man who
Word = who
Most frequent bi-gram = who are
Word = are
Most frequent bi-gram = are not
Word = not
Most frequent bi-gram = not be

 man who are not be
